In [1]:
%run ../processing/read_data.py
%run ../processing/functions.ipynb

from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import time
from PIL import Image

In [2]:
from __future__ import absolute_import, division, print_function

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

1.13.1


In [3]:
path_to_read = '../../data/interpolate_data_low_quality/'
files = []

for r, d, f in os.walk(path_to_read):
    for file in f:
        if '.png' in file:
            files.append(os.path.join(r, file))
files.sort()
np_files = np.asarray(files)

In [4]:
##Verification if each transient has 4 curves
#temp = 'SN000017'
#count = 0
#for f in np_files:
#    if (count == 4):
#        temp = f[40:48]
#        count = 0
#    if ((temp == f[40:48]) & (count < 4)):
#        count += 1
#    elif ((temp != f[40:48]) & (count < 4)):
#        print(f)

In [5]:
#set(types) -> {'IIL', 'IIP', 'II_', 'IIn', 'Ia_', 'Ib_', 'Ibc', 'Ic_'}
conversion = {'IIL' : 5, 'IIP' : 7, 'II_' : 4, 'IIn' : 6, 'Ia_' : 0, 'Ib_' : 1, 'Ibc' : 2, 'Ic_' : 3}

In [6]:
np_grouped = np_files.reshape(round(np_files.shape[0]/4), 4)
class_id = []
for g in np_grouped:
    class_id = class_id + [[g[0][40:48], g[0][49:52]]] 
types = []
for ids in class_id:
    types = types + [conversion[ids[1]]]
set(types)

{0, 1, 2, 3, 4, 5, 6, 7}

In [7]:
astr_objs = []
for i in range(len(np_grouped)):
    astr_objs = astr_objs + [class_id[i]]
    for file in np_grouped[i]:
        img = Image.open(file).convert('L')
        arr = np.array(img)
        astr_objs[-1] = astr_objs[-1] + [arr[5:35,10:56]]
len(astr_objs)


21317

In [8]:
##for ims in astr_objs[0][2:]:
##    print(len(ims[0]))
#
#teste = astr_objs[0][2]
#teste2 = teste[5:35,10:56]
#teste2.shape
#len(teste2[1])

In [9]:
labels = []
images = []
ids = []
for o in astr_objs:
    labels.append((conversion[o[1]])) #converting here to numbers to assure label in training
    images.append(o[2:])
    ids.append(o[0])
labels = np.asarray(labels)
images = np.asarray(images)
ids = np.asarray(ids)

train_n = 18000

train_labels = labels[0:train_n]
test_labels = labels[train_n:]

train_images = images[0:train_n]
test_images = images[train_n:]

train_ids = ids[0:train_n]
test_ids = ids[train_n:]


In [10]:
print(ids.shape, labels.shape, images.shape)

(21317,) (21317,) (21317, 4, 30, 46)


In [11]:
train_images.shape

(18000, 4, 30, 46)

In [12]:
train_images = np.around(train_images/255, decimals = 2)
test_images = np.around(test_images/255, decimals = 2)

# Here starts the training

In [13]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(4, 30, 46)), #aqui mudar o novo shape () 64x40 iu 46x30, x4 talvez 
    #Transform the 2d array into a 1d with mxn 
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(8, activation=tf.nn.softmax) #exit layer, change to the number of class we have ()
])


Instructions for updating:
Colocations handled automatically by placer.


In [14]:
img_rows, img_cols = 30, 46
input_shape = (4, img_rows, img_cols)

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

cnn1 = Sequential()
cnn1.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
cnn1.add(MaxPooling2D(pool_size=(2, 2)))
cnn1.add(Dropout(0.2))

cnn1.add(Flatten())

cnn1.add(Dense(128, activation='relu'))
cnn1.add(Dense(10, activation='softmax'))


Using TensorFlow backend.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [15]:
#from keras.models import Sequential
#from keras.layers import Dense, Dropout, Flatten
#from keras.layers import Conv2D, MaxPooling2D
#
#cnn3 = Sequential()
#cnn3.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
#cnn3.add(MaxPooling2D((2, 2)))
#cnn3.add(Dropout(0.25))
#
#cnn3.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
#cnn3.add(MaxPooling2D(pool_size=(2, 2)))
#cnn3.add(Dropout(0.25))
#
#cnn3.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
#cnn3.add(Dropout(0.4))
#
#cnn3.add(Flatten())
#
#cnn3.add(Dense(128, activation='relu'))
#cnn3.add(Dropout(0.3))
#cnn3.add(Dense(10, activation='softmax'))
#
#cnn3.compile(loss=keras.losses.categorical_crossentropy,
#              optimizer=keras.optimizers.Adam(),
#              metrics=['accuracy'])

In [16]:
#cnn1.compile(optimizer='adam', 
#              loss='sparse_categorical_crossentropy',
#              metrics=['accuracy'])

In [17]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
#cnn1.fit(train_images, train_labels, epochs=10)
#

In [19]:
model.fit(train_images, train_labels, epochs=10)


Epoch 1/10
18000/18000 [==============================] - 11s 595us/sample - loss: 1.3131 - acc: 0.5666
Epoch 2/10
18000/18000 [==============================] - 9s 498us/sample - loss: 1.2899 - acc: 0.5669
Epoch 3/10
18000/18000 [==============================] - 9s 501us/sample - loss: 1.2672 - acc: 0.5693
Epoch 4/10
18000/18000 [==============================] - 9s 495us/sample - loss: 1.1660 - acc: 0.6178
Epoch 5/10
18000/18000 [==============================] - 9s 502us/sample - loss: 1.0757 - acc: 0.6592
Epoch 6/10
18000/18000 [==============================] - 9s 494us/sample - loss: 1.0375 - acc: 0.6728
Epoch 7/10
18000/18000 [==============================] - 9s 505us/sample - loss: 1.0180 - acc: 0.6813
Epoch 8/10
18000/18000 [==============================] - 9s 498us/sample - loss: 1.0204 - acc: 0.6780
Epoch 9/10
18000/18000 [==============================] - 9s 500us/sample - loss: 1.0044 - acc: 0.6908
Epoch 10/10
18000/18000 [==============================] - 9s 493us/samp

In [20]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

print('Test accuracy:', test_acc)

3317/3317 [==============================] - 1s 177us/sample - loss: 0.9978 - acc: 0.7006
Test accuracy: 0.7006331


In [21]:
predictions = model.predict(test_images)

In [22]:
print(predictions[0], '\n',
      'Predicted: ', np.argmax(predictions[0]), ' True one: ', test_labels[0])


[0.13226014 0.04723146 0.01049704 0.05227954 0.67416096 0.015006
 0.05238821 0.01617657] 
 Predicted:  4  True one:  4


In [23]:
##TEMPLATE DE COMO PEGAR O ARQUIVO
#import numpy as np
#from PIL import Image
#
#img = Image.open('SN000017_II_desg.png').convert('L')
#arr = np.array(img)
#arr = arr[5:35,10:56]
## record the original shape
#shape = arr.shape
#
## make a 1-dimensional view of arr
#flat_arr = arr.ravel()
#
## convert it to a matrix
#vector = np.matrix(flat_arr)
#
## do something to the vector
#vector[:,::10] = 128
#
## reform a numpy array of the original shape
#arr2 = np.asarray(vector).reshape(shape)
#
## make a PIL image
#img2 = Image.fromarray(arr2, 'L')
#img2.show()

In [24]:
def plot_value_array(i, predictions_array, true_label):
    predictions_array, true_label = predictions_array[i], true_label[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    thisplot = plt.bar(range(10), predictions_array, color="#777777")
    plt.ylim([0, 1]) 
    predicted_label = np.argmax(predictions_array)
    thisplot[predicted_label].set_color('red')
    thisplot[true_label].set_color('blue')


In [25]:
from sklearn.metrics import confusion_matrix
y_true = test_labels
y_pred = []
for i in predictions:
    y_pred.append(np.argmax(i))

confusion_matrix(y_true, y_pred)

array([[ 630,    0,    0,    0,  176,    0,    0,    0],
       [ 164,    0,    0,    0,   69,    0,    0,    0],
       [  29,    0,    0,    0,   15,    0,    0,    0],
       [  77,    0,    0,    0,   92,    0,    0,    0],
       [ 131,    0,    0,    0, 1694,    0,    0,    0],
       [  57,    0,    0,    0,   19,    0,    0,    0],
       [  19,    0,    0,    0,  111,    0,    0,    0],
       [  12,    0,    0,    0,   22,    0,    0,    0]])

In [26]:
unique, counts = np.unique(labels, return_counts=True)

In [27]:
unique

array([0, 1, 2, 3, 4, 5, 6, 7])

In [28]:
counts

array([ 5087,  1438,   259,  1103, 12027,   425,   789,   189])

In [29]:
conversion

{'IIL': 5,
 'IIP': 7,
 'II_': 4,
 'IIn': 6,
 'Ia_': 0,
 'Ib_': 1,
 'Ibc': 2,
 'Ic_': 3}

In [30]:
y_true.shape

(3317,)

In [31]:
y_pred = np.asarray(y_pred)

In [32]:
conversion

{'IIL': 5,
 'IIP': 7,
 'II_': 4,
 'IIn': 6,
 'Ia_': 0,
 'Ib_': 1,
 'Ibc': 2,
 'Ic_': 3}

In [33]:
class_names = ['Ia','Ib','Ibc','Ic','II','IIL','IIP','IIn']

In [43]:
np.set_printoptions(precision=1)

# Plot non-normalized confusion matrix
plot_confusion_matrix(y_true, y_pred, classes=class_names,
                      dim_x = len(y_true), dim_y = len(y_true),
                      title='Confusion matrix, without normalization')

## Plot normalized confusion matrix
#plot_confusion_matrix(y_true, y_pred, classes=class_names, normalize=True,
#                      title='Normalized confusion matrix')
#
plt.show()

Confusion matrix, without normalization


ValueError: Image size of 331700x331700 pixels is too large. It must be less than 2^16 in each direction.

In [36]:
test_images

array([[[[1. , 1. , 1. , ..., 1. , 1. , 1. ],
         [1. , 1. , 1. , ..., 1. , 1. , 1. ],
         [1. , 1. , 1. , ..., 1. , 1. , 1. ],
         ...,
         [1. , 0.5, 0.5, ..., 1. , 1. , 1. ],
         [0.5, 0.7, 1. , ..., 0.9, 0.9, 0.9],
         [0.9, 1. , 1. , ..., 0.6, 0.5, 0.7]],

        [[1. , 1. , 1. , ..., 1. , 1. , 1. ],
         [1. , 1. , 1. , ..., 1. , 1. , 1. ],
         [1. , 1. , 1. , ..., 1. , 1. , 1. ],
         ...,
         [1. , 1. , 0.9, ..., 1. , 1. , 0.8],
         [1. , 1. , 1. , ..., 1. , 1. , 1. ],
         [1. , 1. , 1. , ..., 1. , 1. , 1. ]],

        [[1. , 1. , 1. , ..., 1. , 1. , 1. ],
         [1. , 1. , 1. , ..., 1. , 1. , 1. ],
         [1. , 1. , 1. , ..., 1. , 1. , 1. ],
         ...,
         [0.8, 1. , 1. , ..., 1. , 1. , 0.9],
         [1. , 1. , 1. , ..., 1. , 1. , 1. ],
         [1. , 1. , 1. , ..., 1. , 1. , 1. ]],

        [[1. , 1. , 1. , ..., 1. , 1. , 1. ],
         [1. , 1. , 1. , ..., 1. , 1. , 1. ],
         [1. , 1. , 1. , ..., 1.

In [38]:
plot_confusion_matrix

<function __main__.plot_confusion_matrix(y_true, y_pred, classes, dim_x, dim_y, normalize=False, title=None, cmap=<matplotlib.colors.LinearSegmentedColormap object at 0x7fd5b6579fd0>)>

In [40]:
help(plot_confusion_matrix)

Help on function plot_confusion_matrix in module __main__:

plot_confusion_matrix(y_true, y_pred, classes, dim_x, dim_y, normalize=False, title=None, cmap=<matplotlib.colors.LinearSegmentedColormap object at 0x7fd5b6579fd0>)
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.



In [41]:
len(y_true)

3317

In [1]:
import pickle

In [6]:
x = pickle.load(open("/home/felipematheus/Documents/tcc/notebooks/data/interpolate_data_img/SN000762_II_desr.png", "rb"))

UnpicklingError: A load persistent id instruction was encountered,
but no persistent_load function was specified.